In [1]:
%load_ext autoreload
%autoreload 2
#import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
import keras
from tqdm import tqdm, tqdm_pandas
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
import csv
warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the df that we created in the other notebook
df_files = pd.read_csv("safe/df_file_final.csv", lineterminator='\n')

In [3]:
## Transform into array from string. Also rename column if a problem occured during loading.
from ast import literal_eval
if("sentences\r" in df_files.columns):
    df_files = df_files.rename(index=str,columns={'sentences\r': 'sentences'})
df_files.sentences = df_files.sentences.apply(literal_eval)

In [4]:
df_files.head(5).sentences[0]

([('Keppel Capital aims to create value and deliver sustainable returns for institutional and retail investors through a range of products including REITs, business trusts, private funds investing in real es tate in Asia, separate accounts and pooled investment vehicles.',
   5.163275694908415),
  ('Keppel Capital is a premier asset manager in Asia.', 10.307097586053025),
  ('Represent Keppel DC REIT in HSE commit tee meetings and annual safety events.',
   4.520748868940497),
  ('The asset managers under Keppel Capital include Keppel REIT Management Limited, Alpha Investment Partners Limited, Keppel Infrastructure Fund Management Pte Ltd and Keppel DC REIT Management P te Ltd.',
   20.267967689101564),
  ('It is the asset management arm of Keppel Corporation Limited (Keppel Corporation), a multinational conglomerate with key businesses in Offshore &',
   10.284593789539135),
  ('billion, Keppel Capital has a diversified portfolio that includes real estate, infrastructur e and data cen

In [5]:
##Method to predict the average sentiment of a list of phrase which are tuple containing (str,float) -> 
##(sentence,weight)
def average(tupple, sentimentAnalyzer):
    sents, imporances = zip(*tupple)
    sents = list(sents)
    importance = list(imporances)
    pred = np.squeeze(sentimentAnalyzer.predict(sents))
    return np.average(pred, weights=imporances)

##Return sentiment final given top_k phrases and company_phrases, with a hyper parameter to sum to a final result.
def find_sentiment(important_sents,sentimentAnalyzer):
    try:
        if important_sents == [-500]:
            return -500
        interest_sent, top_sent = important_sents
        lambda_ = 0.3
        return (1-lambda_)*average(interest_sent,sentimentAnalyzer) + \
            lambda_*average(top_sent, sentimentAnalyzer)
    except:
        return -999

In [6]:
#model = keras.models.load_model('models/my_amazon_model.h5')
analyzser = sentiment_analyzer.SentimentAnalyzer('models/my_model.h5')
tqdm().pandas(desc="Progress:")
df_files.sentiment = df_files.progress_apply(lambda row: np.mean(find_sentiment(row['sentences'],analyzser)),axis=1)

<class 'function'>
Instructions for updating:
dim is deprecated, use axis instead


0it [00:00, ?it/s]
Progress::   6%|▌         | 24/387 [00:16<05:12,  1.16it/s]


 35%|███▌      | 127/361 [00:00<00:00, 247.13it/s]


100%|██████████| 361/361 [00:01<00:00, 244.17it/s]

100%|██████████| 168/168 [00:00<00:00, 273.88it/s]

100%|██████████| 10/10 [00:02<00:00,  4.07it/s]

 40%|████      | 134/335 [00:00<00:01, 146.91it/s]


100%|██████████| 297/297 [00:01<00:00, 240.99it/s]

  8%|▊         | 26/317 [00:00<00:01, 255.75it/s]


100%|██████████| 5/5 [00:00<00:00, 47.49it/s]

100%|██████████| 2/2 [00:00<00:00, 16.75it/s]

 54%|█████▍    | 246/457 [00:00<00:00, 267.46it/s]


100%|██████████| 27/27 [00:00<00:00, 264.60it/s]

100%|██████████| 11/11 [00:00<00:00, 109.13it/s]

  0%|          | 0/5 [00:00<?, ?it/s]


 34%|███▍      | 69/203 [00:00<00:00, 220.88it/s]


Progress:: 100%|██████████| 387/387 [02:05<00:00,  3.08it/s]


In [11]:
df_files

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,0.571920,([(Keppel Capital aims to create value and del...
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,0.350840,([(Company Backg round Keppel is a diversified...
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,0.754812,([(The new SOCAR-Keppel Shipyard will be devel...
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,-999.000000,[-500]
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,0.276819,([(Jurong Shipyard Pte Ltd (Songa Eclipse) Jur...
5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,0.577598,([(2.3.6 West Keppel Fairway That part of the ...
6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,0.632227,"([(Within Keppel O&M, there are three technolo..."
7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,0.537239,([(employees from across the Keppel Group atte...
8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,0.677692,([(Working experience and occupation(s) during...
9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,-999.000000,[-500]


In [12]:
##Get the index of the top and worst document from a dataframe with value filled.
def getSubmission(df):
    """Return : 4 array of index that correspond to worst and best document of each company"""
    dfCleaned = df[df['sentiment']>0.01]
    top_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=False).head(10).id)
    down_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=True).head(10).id)
    top_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=False).head(10).id)
    down_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=True).head(10).id)
    return top_keppel,down_keppel, top_prud, down_prud

In [21]:
def createSubCSV(topId_k,downId_k,topId_p,downId_p):
    """Create a CSV submission from the index of top/worst document of each company"""
    with open("sub.csv",'w',newline='') as csvfile:
        spamwritter = csv.writer(csvfile)
        spamwritter.writerow(['id','rank'])
        for i in range(len(df_files.sentiment)):
            if(i in downId_k):
                spamwritter.writerow([i,(downId_k.index(i)-10)])
            elif(i in topId_k):
                spamwritter.writerow([i,(10-topId_k.index(i))])
            elif(i in downId_p):
                spamwritter.writerow([i,(downId_p.index(i)-10)])
            elif(i in topId_p):
                spamwritter.writerow([i,(10-topId_p.index(i))])
            else:
                spamwritter.writerow([i,0])

In [25]:
top_k,down_k, top_p, down_p = getSubmission(df_files)
createSubCSV(top_k,down_k, top_p, down_p)